In [1]:
# Import dependencies
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re # to work with regex
import time # to add delays between requests

## Initial scraping

In [2]:
# chromedriver_autoinstaller.install()
# # Launch browser
# driver = webdriver.Chrome()
# # Visit listing.ca
# url = "https://toronto.listing.ca/"
# driver.get(url)

In [3]:
# # Scrape the website
# html = driver.page_source

# # Create a BeautifulSoup object from the scraped HTML
# soup = BeautifulSoup(html, 'html.parser')

In [4]:
# # List of listings in the html soup, found using class sl
# listings_html = soup.find_all('div', 'sl')
# # Print first element of the list
# listings_html[0]


In [5]:
# # Loop through listings and append the clean info a list of dictionaries
# listings_list = []
# for item in listings_html:
#     # Create a dictionary containing the info for a listing
#     listing = {
#         'url': item.find('div', 'slt_address').a['href'],
#         'address': item.find('div', 'slt_address').text,
#         'price': item.find('div', 'slt_price').text,
#         'baths': item.find('div', 'slt_baths').text,
#         'beds': item.find('div', 'slt_beds').text,
#         'neighbourhood': item.find_all('na2')[1].text
#     }
#     # Append to the list
#     listings_list.append(listing)
# # Check results
# listings_list[0:3]

In [6]:
# # Create dataframe with info from first page's listings
# df = pd.DataFrame(listings_list)
# df.head()

In [7]:
# # Extract mls id from url column
# df['mls_id'] = [re.search('[A-Z]\\d{7}', url).group() for url in df['url']]
# df.head()

In [8]:
# # Quit browsing session
# driver.quit()

## Functions and page loops

In [9]:
# Define function to scrape a single page,
# @arg html: scraped html
# @return list of listing dictionaries
def get_listing_info(html):
    # Create a BeautifulSoup object from the scraped HTML
    soup = BeautifulSoup(html, 'html.parser')
    # List of listings in the html soup, found using class sl
    listings_html = soup.find_all('div', 'sl')
    # Empty list to store listing dicts
    listings_list = []
    for item in listings_html:
        # Create a dictionary containing the info for a listing
        listing = {
            'url': item.find('div', 'slt_address').a['href'],
            'address': item.find('div', 'slt_address').text,
            'price': item.find('div', 'slt_price').text,
            'baths': item.find('div', 'slt_baths').text,
            'beds': item.find('div', 'slt_beds').text,
            'neighbourhood': item.find_all('na2')[1].text
        }
        # Extract mls_id from url
        listing['mls_id'] = re.search('[A-Z]\\d{7}', listing['url']).group()
        # Append to the list
        listings_list.append(listing)
    # Return expanded listings_list
    return listings_list

In [10]:
chromedriver_autoinstaller.install()
# Launch browser
driver = webdriver.Chrome()

In [11]:
# Loop to collect the listing info and click to the next page for 3 total pages
listing_info_list = []

for i in range(1, 4):
    url = f'https://listing.ca/mls/?.cy.........{i}..$'
    driver.get(url)
    listing_info_list.extend(get_listing_info(driver.page_source))
    time.sleep(5) 
    # driver.find_element(By.PARTIAL_LINK_TEXT, 'Next').click()

In [12]:
# Create dataframe with info from first page's listings
df = pd.DataFrame(listing_info_list)
df.head()

,url,address,price,baths,beds,neighbourhood,mls_id
0,https://toronto.listing.ca/286-main-st-911.E80...,286 Main St 911,"$619,900 CAD",1 baths,1+1 beds,East End-Danforth,E8018446
1,https://toronto.listing.ca/18-willowlea-dr.E73...,18 Willowlea Dr,"$1,699,900 CAD",0 baths,0 beds,Highland Creek,E7353624
2,https://toronto.listing.ca/5-kingsbury-cres.E7...,5 Kingsbury Cres,"$15,000,000 CAD",1 baths,2 beds,Birchcliffe-Cliffside,E7292902
3,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,"$529,000 CAD",1 baths,1 beds,Waterfront Communities C1,C7266728
4,https://toronto.listing.ca/10-park-lawn-rd-140...,10 Park Lawn Rd 1408,"$624,900 CAD",1 baths,1+1 beds,Mimico,W7239426


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   url            60 non-null     object
 1   address        60 non-null     object
 2   price          60 non-null     object
 3   baths          60 non-null     object
 4   beds           60 non-null     object
 5   neighbourhood  60 non-null     object
 6   mls_id         60 non-null     object
dtypes: object(7)
memory usage: 3.4+ KB


In [14]:
# Quit browsing session
driver.quit()